In [106]:
import pandas as pd
import json
import plotly.graph_objects as go
import numpy as np

In [65]:
covid = pd.read_csv('https://api.covidtracking.com/v1/states/current.csv')

In [66]:
with open('state_lookup.json') as f:
    code_lookup = json.load(f)
    state_lookup = {y:x for x,y in code_lookup.items()}

In [67]:
population = pd.read_csv('state_populations.csv')

In [68]:
population['code']=[code_lookup[s] for s in population['State']]

In [69]:
def pop_lookup(state):
    try:
        return pop_dict[state]
    except:
        return None

In [70]:
covid['population']=[pop_lookup(s) for s in covid['state']]

In [71]:
covid.dropna(subset=['population'], inplace=True)

In [74]:
covid.columns

Index(['date', 'state', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResults', 'hospitalizedCurrently', 'hospitalizedCumulative',
       'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently',
       'onVentilatorCumulative', 'recovered', 'dataQualityGrade',
       'lastUpdateEt', 'dateModified', 'checkTimeEt', 'death', 'hospitalized',
       'dateChecked', 'totalTestsViral', 'positiveTestsViral',
       'negativeTestsViral', 'positiveCasesViral', 'deathConfirmed',
       'deathProbable', 'totalTestEncountersViral', 'totalTestsPeopleViral',
       'totalTestsAntibody', 'positiveTestsAntibody', 'negativeTestsAntibody',
       'totalTestsPeopleAntibody', 'positiveTestsPeopleAntibody',
       'negativeTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'positiveTestsPeopleAntigen', 'totalTestsAntigen',
       'positiveTestsAntigen', 'fips', 'positiveIncrease', 'negativeIncrease',
       'total', 'totalTestResultsSource', 'totalTestResultsIncrease', 'posNe

In [82]:
df = covid.filter(['state','positive','death','population'])
df['per_capita_positive']=[(pos/pop)*100000 for pos,pop in zip(df['positive'],df['population'])]
df['per_capita_death']=[(death/pop)*100000 for death,pop in zip(df['death'],df['population'])]
df['odds_positive']=[100000/pcp for pcp in df['per_capita_positive']]
df['odds_death']=[100000/pcd for pcd in df['per_capita_death']]

In [2]:
import plotly.graph_objects as go

In [107]:
np.round(8928.32)

8928.0

In [150]:
fig = go.Figure()

examples = [2, 10, 50, 100, 200, 500, 1000, 10000]

fig.add_trace(go.Scatter(
    name='Example',
    x=examples,
    y=[100000/ex for ex in examples],
    mode='lines+markers',
    hovertemplate = '<b>Example</b><br><b>Per capita:</b> %{x} per 100,000 People<br><b>Odds:</b> 1 in %{y} People<extra></extra>',
    opacity=0.5,
    line = dict(color='slateblue')
))

fig.add_trace(go.Scatter(
    name='Positive Cases',
    x=[np.round(n) for n in df['per_capita_positive']],
    y=[np.round(n) for n in df['odds_positive']],
    mode='markers',
    text=[state_lookup[s] for s in df['state']],
    hovertemplate = 'Cases in <b>%{text}</b><br><b>Per capita:</b> %{x} per 100,000 People<br><b>Odds:</b> 1 in %{y} People<extra></extra>',
    line = dict(color='purple'),
    marker=dict(
        size=df['population']/500000
    )
))

fig.add_trace(go.Scatter(
    name='Deaths',
    x=[np.round(n) for n in df['per_capita_death']],
    y=[np.round(n) for n in df['odds_death']],
    mode='markers',
    text=[state_lookup[s] for s in df['state']],
    hovertemplate = 'Deaths in <b>%{text}</b><br><b>Per capita:</b> %{x} per 100,000 People<br><b>Odds:</b> 1 in %{y} People<extra></extra>',
    line = dict(color='red'),
    marker=dict(
        size=df['population']/500000
    )
))

fig.update_xaxes(type="log", showspikes=True)
fig.update_yaxes(type="log", autorange="reversed", showspikes=True)
fig.update_layout(
    template='plotly_white',
    xaxis_title="Per Capita: <b>X</b> per 100,000 People",
    yaxis_title="Odds: 1 in <b>Y</b> People",
    title="Per Capita vs. Odds: Positive Cases and Deaths"
)

fig.add_annotation(
        x=1,
        y=1,
        text="Case & death data from <a href='https://covidtracking.com' target='_blank'>covidtracking.com</a><br>Population data from <a href='https://worldpopulationreview.com/states' target='_blank'>World Population Review</a>")
fig.update_annotations(dict(
        xref="x",
        yref="y",
        showarrow=False,
        bordercolor="gray",
        borderwidth=2,
        borderpad=6,
        bgcolor="white"
))

fig.write_html("covid_per_capita_vs_odds.html")

fig.show()